In [1]:
import json
from itertools import cycle, islice
from shapely.geometry import LineString, Polygon
import numpy as np
from basepolygon_class import GoalPolygon
from scipy.spatial import distance
from itertools import combinations
from sklearn.cluster import AffinityPropagation
from scipy.spatial.distance import pdist


In [2]:
place = GoalPolygon('Жилой комплекс «Одинцово-1»')
v = place.polygon
s = v.simplify(10, preserve_topology=True)
coor_list = (list(s.boundary.coords[0:len(s.boundary.coords) - 1]))
poly = Polygon(s)
print(list(poly.boundary.coords))


/opt/anaconda3/envs/tests/lib/python3.10/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/opt/anaconda3/envs/tests/lib/python3.10/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


[(390638.30219235073, 6169388.159907748), (390729.22685795417, 6169068.278793206), (390688.94853554666, 6169051.093999996), (390779.71763066336, 6168708.404413646), (391376.94128866255, 6168878.908844141), (391679.81934692746, 6169195.94232304), (391371.67693937925, 6169504.334368904), (391167.0485137885, 6169300.755649202), (390888.987181012, 6169257.282553047), (390787.08946967893, 6169633.4398970865), (390723.15643382387, 6169615.510903369), (390713.85588372935, 6169648.534112789), (390648.1477669974, 6169630.105088735), (390710.60008595965, 6169408.428049387), (390638.30219235073, 6169388.159907748)]


/opt/anaconda3/envs/tests/lib/python3.10/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [3]:
def vector_search(point_list, key):
    cr = []
    n = list(islice(cycle(range(len(point_list))), 0, len(point_list) + 2))
    for i in range(len(point_list)):
        v_one = [point_list[n[i]][0] - point_list[n[i + 1]][0], point_list[n[i]][1] - point_list[n[i + 1]][1], 0]
        v_two = [point_list[n[i + 1]][0] - point_list[n[i + 2]][0], point_list[n[i + 1]][1] - point_list[n[i + 2]][1],
                 0]
        v_o_norm, v_t_norm = v_one / np.linalg.norm(v_one), v_two / np.linalg.norm(v_two)
        if np.cross(v_o_norm, v_t_norm)[2] < key:
            cr.append(n[i + 1])
        else:
            pass
    cross = [*cr, cr[0] + cr[-1] + 1]
    double_points = [*range(len(point_list)), *range(len(point_list))]
    segments = [double_points[cross[c]:cross[c + 1]] for c in range(len(cross) - 1)]
    return segments


def clusterize(poly_edges):
    lines = []
    vectors_list = []
    key_vals = vector_search(poly_edges.boundary.coords[0:len(s.boundary.coords) - 1], 0.7)
    vertices = poly_edges.boundary.coords[0:len(s.boundary.coords)]
    for k in range(len(vertices) - 1):
        lines.append(LineString([vertices[k], vertices[k + 1]]))
    for i in key_vals:
        l=[]
        for ii in i:
            vect = [lines[ii].coords[0][0] - lines[ii].coords[1][0],
                    lines[ii].coords[0][1] - lines[ii].coords[1][1]]
            l.append(vect)
            #l.append(list(vect / np.linalg.norm(vect)))
        vectors_list.append(l)

    return vectors_list


In [152]:
v = clusterize(poly)
print(v)

[[[40.27832240751013, 17.184793210588396], [-90.76909511670237, 342.689586349763], [-597.2236579991877, -170.50443049520254]], [[-302.8780582649051, -317.03347889892757], [308.1424075482064, -308.39204586390406], [204.62842559075216, 203.57871970254928]], [[278.06133277650224, 43.473096154630184]], [[101.89771133306203, -376.15734403952956], [63.9330358550651, 17.928993717767298]], [[9.30055009451462, -33.0232094200328], [65.70811673195567, 18.42902405373752], [-62.45231896225596, 221.67703934758902]], [[72.29789360892028, 20.2681416394189], [-90.92466560343746, 319.88111454155296]]]


In [121]:
from scipy.linalg import svd
import numpy as np
A = np.asarray(v)

/var/folders/l7/52t7xngd2973k8x9_y55rmyh0000gn/T/ipykernel_11979/2796483893.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  A = np.asarray(v)


Пример с одной матрицей

In [43]:
a = np.asarray(A[0])

In [44]:
U, s, Vh=svd(a)

Получить Sigma из s Хуй знает зачем сайпи говорят что это полезно

In [45]:
sigma = np.zeros(a.shape)
for i in range(min(a.shape)):
           sigma[i, i] = s[i]

А точно, как минимум для обратной проверки, "B" должно быть равно "a"

In [46]:
B = U @ sigma @ Vh

In [47]:
a

array([[  40.27832241,   17.18479321],
       [ -90.76909512,  342.68958635],
       [-597.223658  , -170.5044305 ]])

In [37]:
B

array([[  40.27832241,   17.18479321],
       [ -90.76909512,  342.68958635],
       [-597.223658  , -170.5044305 ]])

Функция итерирующая список матриц и вычисляющая свд для каждой

In [167]:
def csvd(aa):
    svdd=[]
    vh = []
    for a_ in aa:
        a=np.asarray(a_)
        #print(f"A:\n{a}")
        U, s, Vh=svd(a)
        #print("U")
        #print(U)
        sigma = np.zeros(a.shape)
        #print("s")
        #print(s)
        #print("Vh")
        #print(Vh)
        for i in range(min(a.shape)):
            sigma[i, i] = s[i]
        #print("Sigma")
        #print(sigma)
        svdd.append((a, U.tolist(), s.tolist(), Vh, sigma))
        vh.append(vh)

    return np.asarray(vh)

In [ ]:


n=csvd(A)
print(n)

In [159]:
import pandas as pd
df = pd.DataFrame(data = d)



1.0471975511965976
nan
2.0943951023931953
2.0943951023931957
2.078901238663254
2.0943951023931957
next
nan
1.047197551196598
2.0943951023931957
2.0943951023931957
nan
2.0943951023931957
next
2.0943951023931953
2.0943951023931957
1.0471975511965976
1.9725443091137271
2.0943951023931957
1.9587076350192205
next
2.0943951023931957
2.0943951023931957
1.9725443091137271
1.0471975511965979
2.0943951023931957
1.0472929986120654
next
2.078901238663254
nan
2.0943951023931957
2.0943951023931957
1.0471975511965979
2.0943951023931957
next
2.0943951023931957
2.0943951023931957
1.9587076350192205
1.0472929986120654
2.0943951023931957
1.0471975511965976
next


/var/folders/l7/52t7xngd2973k8x9_y55rmyh0000gn/T/ipykernel_11979/375021768.py:5: RuntimeWarning: invalid value encountered in arccos
  return np.arccos((np.trace(b)-1)/2)


In [134]:
import torch
for nn in n:
    n_t = torch.tensor(nn)
    for mm in n:
        m_t = torch.tensor(mm)
        print (np.asarray(torch.dist(n_t, m_t, p=2)).tolist())
    print ('next')

0.0
2.723350242496944
2.0
1.9999999999999998
1.9864767237547665
2.0
next
2.723350242496944
0.0
2.0
1.9999999999999998
2.475056726357569
2.0
next
2.0
2.0
0.0
1.8878853195026353
1.9999999999999998
1.8743059913154232
next
1.9999999999999998
1.9999999999999998
1.8878853195026353
0.0
1.9999999999999998
0.01818374702279813
next
1.9864767237547665
2.475056726357569
1.9999999999999998
1.9999999999999998
0.0
2.0
next
2.0
2.0
1.8743059913154232
0.01818374702279813
2.0
0.0
next
